In [1]:
#Bruno Marra de Melo - 3029

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
pd.options.display.float_format = "{:.2f}".format

In [2]:
df = pd.read_csv('/code/src/Atividades/atividade5/Dataset.csv', index_col=False, squeeze=True)

#unit price veio como string, realizando a conversão para float
df["UnitPrice"] = [x.replace(",", ".") for x in df["UnitPrice"]]
df["UnitPrice"] = df["UnitPrice"].astype(float)

#tira os espaços em branco do começo e do final
df['Description'] = df['Description'].str.strip()

#remove linhas sem InvoiceNo
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

#converte invoiceno para string
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.00,United Kingdom


In [18]:
#pega só frança
france = df[df['Country'] == 'France']

#agrupa por invoiceno e description para somar as quantidades e saber o número de transações
df_grouped = france.groupby(['InvoiceNo', 'Description'])['Quantity'].sum()

#converte os indices em linhas e colunas
cesta_franca = df_grouped.unstack(fill_value=0)

#Troca o índice para InvoiceNo
cesta_franca.reset_index().set_index('InvoiceNo')

def encode_units(x):
    if x <= 0:
        return 0
    else:
        return 1
    
cesta_franca = cesta_franca.applymap(encode_units)
cesta_franca.drop('POSTAGE', inplace=True, axis=1)
cesta_franca.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
frequent_itemsets = apriori(cesta_franca, min_support=0.03, use_colnames=True)
support = association_rules(frequent_itemsets, metric="support", min_threshold=0.05)
support.sort_values(by='support').tail(3)

#a) Qual regra possui o maior suporte?
    #Logo, como podemos observar, duas regras empatam com suporte de 0.10
    #(SET/6 RED SPOTTY PAPER PLATES)	(SET/6 RED SPOTTY PAPER CUPS
    #(SET/6 RED SPOTTY PAPER CUPS)	(SET/6 RED SPOTTY PAPER PLATES)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
25,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.12,0.15,0.09,0.76,5.22,0.07,3.55
32,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.11,0.12,0.10,0.96,8.20,0.09,22.07
33,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.12,0.11,0.10,0.89,8.20,0.09,8.02


In [62]:
conffidence = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.98)
conffidence.sort_values(by='confidence').tail(5)

#b) Qual regra possui maior confiança?
    #Logo, como podemos observar, várias regras possuem confiança de 1.00

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,"(SET/20 RED RETROSPOT PAPER NAPKINS, MINI PAIN...",(SET/6 RED SPOTTY PAPER PLATES),0.03,0.11,0.03,1.00,9.22,0.03,inf
2,"(SET/6 RED SPOTTY PAPER PLATES, MINI PAINT SET...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.03,0.11,0.03,1.00,8.87,0.03,inf
1,"(SET/20 RED RETROSPOT PAPER NAPKINS, MINI PAIN...",(SET/6 RED SPOTTY PAPER CUPS),0.03,0.12,0.03,1.00,8.54,0.03,inf
10,"(SET/6 RED SPOTTY PAPER PLATES, SET OF 9 BLACK...",(SET/6 RED SPOTTY PAPER CUPS),0.03,0.12,0.03,1.00,8.54,0.03,inf
22,"(PACK OF 6 SKULL PAPER CUPS, PACK OF 6 SKULL P...",(SET/6 RED SPOTTY PAPER PLATES),0.03,0.11,0.03,1.00,9.22,0.03,inf


In [63]:
lift = association_rules(frequent_itemsets, metric="lift", min_threshold=25)
lift.sort_values(by='lift').tail(3)

#c) Qual regra possui maior lift?
    #Logo, como podemos observar, duas regras empatam com lift de 28.81
    #(SET/6 RED SPOTTY PAPER PLATES, PACK OF 20 SKU...	(PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...
    #(PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...	(SET/6 RED SPOTTY PAPER PLATES, PACK OF 20 SKU...

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
7,"(PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...","(PACK OF 6 SKULL PAPER CUPS, SET/6 RED SPOTTY ...",0.03,0.03,0.03,0.87,26.89,0.03,7.74
5,"(PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...","(SET/6 RED SPOTTY PAPER PLATES, PACK OF 20 SKU...",0.03,0.03,0.03,0.87,28.81,0.03,7.76
2,"(SET/6 RED SPOTTY PAPER PLATES, PACK OF 20 SKU...","(PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...",0.03,0.03,0.03,1.00,28.81,0.03,inf


In [64]:
support = association_rules(frequent_itemsets, metric="support", min_threshold=0.02)
len(support[support['confidence'] >= 0.8])
#d) Quantas regras possuem suporte maior ou igual a 0.02 e confiança maior ou igual a 0.8?

91

In [65]:
len(support[(support['confidence'] >= 0.8) & (support['lift'] >= 4)])
#e) Quantas regras possuem lift maior ou igual a 4 e confiança maior ou igual a 0.8?

91

In [66]:
#f) Quais produtos você recomendaria colocar próximos se você fosse organizar um supermercado físico?

#Essa relação eu concerteza levaria em consideração, visto que o lift e a confiança são muito altas, e o suporte
#não é tão baixo comparado a outras relações.
#(SET/6 RED SPOTTY PAPER PLATES, PACK OF 20 SKU...	(PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...

#Bem como também usaria essa relação abaixo, dado que o suporte é o mais alto e possuem um lift médio e uma
#confiança também relativamente alta
#(SET/6 RED SPOTTY PAPER PLATES)	(SET/6 RED SPOTTY PAPER CUPS)
#(SET/6 RED SPOTTY PAPER CUPS)	(SET/6 RED SPOTTY PAPER PLATES)

In [68]:
#pega só alemanha
germany = df[df['Country'] == 'Germany']

#agrupa por invoiceno e description para somar as quantidades e saber o número de transações
df_grouped = germany.groupby(['InvoiceNo', 'Description'])['Quantity'].sum()

#converte os indices em linhas e colunas
cesta_alemanha = df_grouped.unstack(fill_value=0)

#Troca o índice para InvoiceNo
cesta_alemanha.reset_index().set_index('InvoiceNo')

def encode_units(x):
    if x <= 0:
        return 0
    else:
        return 1
    
cesta_alemanha = cesta_alemanha.applymap(encode_units)
cesta_alemanha.drop('POSTAGE', inplace=True, axis=1)
cesta_alemanha.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
frequent_itemsets = apriori(cesta_alemanha, min_support=0.03, use_colnames=True)
support = association_rules(frequent_itemsets, metric="support", min_threshold=0.05)
support.sort_values(by='support').tail(3)

#a) Qual regra possui o maior suporte?
    #Logo, como podemos observar, duas regras empatam com suporte de 0.10
    #(ROUND SNACK BOXES SET OF 4 FRUITS)	(ROUND SNACK BOXES SET OF4 WOODLAND)
    #(ROUND SNACK BOXES SET OF4 WOODLAND)	(ROUND SNACK BOXES SET OF 4 FRUITS)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(PLASTERS IN TIN WOODLAND ANIMALS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.10,0.19,0.06,0.54,2.91,0.04,1.77
4,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.12,0.19,0.10,0.83,4.49,0.08,4.89
5,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4 FRUITS),0.19,0.12,0.10,0.54,4.49,0.08,1.90


In [74]:
conffidence = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
conffidence.sort_values(by='confidence').tail(1)

#b) Qual regra possui maior confiança?
    #Logo, como podemos observar, a regra possui 0.88 de confiança
    #(SET/6 RED SPOTTY PAPER CUPS)	(SET/6 RED SPOTTY PAPER PLATES)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.04,0.04,0.03,0.88,20.29,0.03,7.66


In [77]:
lift = association_rules(frequent_itemsets, metric="lift", min_threshold=20)
lift.sort_values(by='lift').tail(3)

#c) Qual regra possui maior lift?
    #Logo, como podemos observar, duas regras empatam com lift de 20.29
    #(SET/6 RED SPOTTY PAPER PLATES)	(SET/6 RED SPOTTY PAPER CUPS)
    #(SET/6 RED SPOTTY PAPER CUPS)	(SET/6 RED SPOTTY PAPER PLATES)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.04,0.04,0.03,0.81,20.29,0.03,4.99
1,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.04,0.04,0.03,0.88,20.29,0.03,7.66


In [ ]:
#Faça o mesmo procedimento para a Alemanha (Country=’Germany’). 
#Existe alguma similaridade nas principais regras? 
#Quais produtos você recomendaria colocar próximos se você fosse organizar um supermercado físico?

#Existem sim similaridades, como podemos observar que esses produtos de "paper plates" apareceram nos dois países
#com alto lift e confiança. Portanto, da pra se afirmar com alto grau de assertividade de que muito provavelmente
#esses produtos dessa linha são bastante comprados em conjunto e, poderiam ser colocados próximos em um supermercado.